In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
df = pd.read_csv('cleaned.csv')
df.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price,CustomerName,Region,SignupDate,ProductName,Category,Brand,Product,Month,Hour
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,8,12
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,5,22
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,4,7
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,3,22
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,3,15


### Since we are only allowed to use Product.csv and Customers.csv

In [22]:
df = df.drop(columns=['TransactionID', 'TransactionDate', 'Quantity', 'TotalValue'])

In [24]:
df.head()

,CustomerID,ProductID,Price,CustomerName,Region,SignupDate,ProductName,Category,Brand,Product,Month,Hour
0,C0199,P067,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,8,12
1,C0146,P067,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,5,22
2,C0127,P067,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,4,7
3,C0087,P067,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,3,22
4,C0070,P067,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,3,15


In [64]:
df = df.drop(columns = ['Month1', 'Month2', 'Month3'])

In [68]:
df[df.CustomerID == 'C0087']

,CustomerID,ProductID,Price,CustomerName,Region,SignupDate,ProductName,Category,Brand,Product,Month,Hour,TotalPrice,MostCategory,Category1,Category2,Product1,Product2,Product3
3,C0087,P067,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,Summer,22,2077.19,Books,Electronics,Books,Speaker,Smartphone,Textbook
24,C0087,P057,239.70,Travis Campbell,South America,2024-04-11,ActiveWear Smartphone,Electronics,ActiveWear,Smartphone,Summer,3,2077.19,Books,Electronics,Books,Speaker,Smartphone,Textbook
38,C0087,P049,147.95,Travis Campbell,South America,2024-04-11,TechPro Textbook,Books,TechPro,Textbook,Summer,8,2077.19,Books,Electronics,Books,Speaker,Smartphone,Textbook
111,C0087,P062,374.16,Travis Campbell,South America,2024-04-11,HomeSense Novel,Books,HomeSense,Novel,Summer,23,2077.19,Books,Electronics,Books,Speaker,Smartphone,Textbook
367,C0087,P030,277.86,Travis Campbell,South America,2024-04-11,ActiveWear Cookbook,Books,ActiveWear,Cookbook,Summer,18,2077.19,Books,Electronics,Books,Speaker,Smartphone,Textbook
466,C0087,P059,303.20,Travis Campbell,South America,2024-04-11,SoundWave Jeans,Clothing,SoundWave,Jeans,Summer,9,2077.19,Books,Electronics,Books,Speaker,Smartphone,Textbook
779,C0087,P029,433.64,Travis Campbell,South America,2024-04-11,TechPro Headphones,Electronics,TechPro,Headphones,Summer,20,2077.19,Books,Electronics,Books,Speaker,Smartphone,Textbook


In [25]:
df['TotalPrice'] = df.groupby('CustomerID')['Price'].transform('sum')

In [49]:

df.groupby('CustomerID')['Category'].value_counts()


CustomerID  Category   
C0001       Electronics    3
            Books          1
            Home Decor     1
C0002       Clothing       2
            Home Decor     2
                          ..
C0199       Home Decor     2
C0200       Clothing       2
            Books          1
            Electronics    1
            Home Decor     1
Name: count, Length: 567, dtype: int64

In [51]:
# Compute top purchased categories separately
df['Category1'] = df.groupby('CustomerID')['Category'].transform(lambda x: x.value_counts().index[0])
df['Category2'] = df.groupby('CustomerID')['Category'].transform(
    lambda x: x.value_counts().index[1] if len(x.value_counts()) > 1 else x.value_counts().index[0]
)

In [73]:
df['Brand1'] = df.groupby('CustomerID')['Brand'].transform(lambda x: x.value_counts().index[0])
df['Brand2'] = df.groupby('CustomerID')['Brand'].transform(lambda x: x.value_counts().index[1] if len(x.value_counts()) > 1 else x.value_counts().index[0])

In [59]:
df['Product1'] = df.groupby('CustomerID')['Product'].transform(lambda x: x.value_counts().index[0])
df['Product2'] = df.groupby('CustomerID')['Product'].transform(
    lambda x: x.value_counts().index[1] if len(x.value_counts()) > 1 else x.value_counts().index[0])
df['Product3'] = df.groupby('CustomerID')['Product'].transform(
    lambda x: x.value_counts().index[2] if len(x.value_counts()) > 2 else x.value_counts().index[0])

In [74]:
df.head()

,CustomerID,ProductID,Price,CustomerName,Region,SignupDate,ProductName,Category,Brand,Product,...,Hour,TotalPrice,MostCategory,Category1,Category2,Product1,Product2,Product3,Brand1,Brand2
0,C0199,P067,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,...,12,1002.44,Electronics,Electronics,Home Decor,Speaker,Art,Rug,ComfortLiving,HomeSense
1,C0146,P067,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,...,22,1346.48,Electronics,Electronics,Home Decor,Speaker,Rug,Textbook,TechPro,ComfortLiving
2,C0127,P067,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,...,7,1926.28,Electronics,Electronics,Home Decor,Speaker,Art,Cookbook,SoundWave,ComfortLiving
3,C0087,P067,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,...,22,2077.19,Books,Electronics,Books,Speaker,Smartphone,Textbook,ActiveWear,TechPro
4,C0070,P067,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,...,15,1122.26,Books,Books,Electronics,Speaker,Textbook,Cookbook,ComfortLiving,TechPro


In [67]:

df['SignupDate']=pd.to_datetime(df['SignupDate'])
df['Month']=df['SignupDate'].dt.month     
df['Month'] = np.where(df['Month'].isin([4, 5, 6, 7, 8, 9]), 'Summer', 'Winter')


In [75]:
df['Time'] = np.where(df['Hour'].isin([0,1,2,3,4, 5, 6, 7, 8, 9,10,11]), 'Day', 'Night')

In [76]:
df.head()

,CustomerID,ProductID,Price,CustomerName,Region,SignupDate,ProductName,Category,Brand,Product,...,TotalPrice,MostCategory,Category1,Category2,Product1,Product2,Product3,Brand1,Brand2,Time
0,C0199,P067,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,...,1002.44,Electronics,Electronics,Home Decor,Speaker,Art,Rug,ComfortLiving,HomeSense,Night
1,C0146,P067,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,...,1346.48,Electronics,Electronics,Home Decor,Speaker,Rug,Textbook,TechPro,ComfortLiving,Night
2,C0127,P067,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,...,1926.28,Electronics,Electronics,Home Decor,Speaker,Art,Cookbook,SoundWave,ComfortLiving,Day
3,C0087,P067,300.68,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,...,2077.19,Books,Electronics,Books,Speaker,Smartphone,Textbook,ActiveWear,TechPro,Night
4,C0070,P067,300.68,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,ComfortLiving,Speaker,...,1122.26,Books,Books,Electronics,Speaker,Textbook,Cookbook,ComfortLiving,TechPro,Night


In [105]:
df.columns

Index(['CustomerID', 'ProductID', 'Price', 'CustomerName', 'Region',
       'SignupDate', 'ProductName', 'Category', 'Brand', 'Product', 'Month',
       'Hour', 'TotalPrice', 'MostCategory', 'Category1', 'Category2',
       'Product1', 'Product2', 'Product3', 'Brand1', 'Brand2', 'Time'],
      dtype='object')

In [106]:
new = df[['CustomerID', 'Region', 'Brand1', 'Brand2', 'Month',
       'Time', 'TotalPrice', 'Category1', 'Category2',
       'Product1', 'Product2', 'Product3']]
new.head()

,CustomerID,Region,Brand1,Brand2,Month,Time,TotalPrice,Category1,Category2,Product1,Product2,Product3
0,C0199,Europe,ComfortLiving,HomeSense,Winter,Night,1002.44,Electronics,Home Decor,Speaker,Art,Rug
1,C0146,Asia,TechPro,ComfortLiving,Summer,Night,1346.48,Electronics,Home Decor,Speaker,Rug,Textbook
2,C0127,Europe,SoundWave,ComfortLiving,Summer,Day,1926.28,Electronics,Home Decor,Speaker,Art,Cookbook
3,C0087,South America,ActiveWear,TechPro,Summer,Night,2077.19,Electronics,Books,Speaker,Smartphone,Textbook
4,C0070,Europe,ComfortLiving,TechPro,Winter,Night,1122.26,Books,Electronics,Speaker,Textbook,Cookbook


In [107]:
new.shape

(1000, 12)

In [108]:
new = new.drop_duplicates(subset='CustomerID')
print(new.shape)
new.head()
     


(199, 12)


,CustomerID,Region,Brand1,Brand2,Month,Time,TotalPrice,Category1,Category2,Product1,Product2,Product3
0,C0199,Europe,ComfortLiving,HomeSense,Winter,Night,1002.44,Electronics,Home Decor,Speaker,Art,Rug
1,C0146,Asia,TechPro,ComfortLiving,Summer,Night,1346.48,Electronics,Home Decor,Speaker,Rug,Textbook
2,C0127,Europe,SoundWave,ComfortLiving,Summer,Day,1926.28,Electronics,Home Decor,Speaker,Art,Cookbook
3,C0087,South America,ActiveWear,TechPro,Summer,Night,2077.19,Electronics,Books,Speaker,Smartphone,Textbook
4,C0070,Europe,ComfortLiving,TechPro,Winter,Night,1122.26,Books,Electronics,Speaker,Textbook,Cookbook


In [109]:
new.columns

Index(['CustomerID', 'Region', 'Brand1', 'Brand2', 'Month', 'Time',
       'TotalPrice', 'Category1', 'Category2', 'Product1', 'Product2',
       'Product3'],
      dtype='object')

In [110]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [111]:
categorical_cols = ['Region', 'Brand1', 'Brand2', 'Month', 'Time',
       'Category1', 'Category2', 'Product1', 'Product2',
       'Product3']

# Define numerical columns (excluding CustomerID & SignupDate)
numerical_cols = ['TotalPrice']

trf1 = ColumnTransformer([
    ('num', MinMaxScaler(), numerical_cols),  # Scale numerical features
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)  # One-hot encode categorical features
], remainder='drop')

trf1.set_output(transform='pandas')

ColumnTransformer(transformers=[('num', MinMaxScaler(), ['TotalPrice']),
                                ('cat',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse_output=False),
                                 ['Region', 'Brand1', 'Brand2', 'Month', 'Time',
                                  'Category1', 'Category2', 'Product1',
                                  'Product2', 'Product3'])])

In [112]:
df_encoded = trf1.fit_transform(new)

In [113]:
df_encoded

,num__TotalPrice,cat__Region_Asia,cat__Region_Europe,cat__Region_North America,cat__Region_South America,cat__Brand1_ActiveWear,cat__Brand1_BookWorld,cat__Brand1_ComfortLiving,cat__Brand1_HomeSense,cat__Brand1_SoundWave,...,cat__Product3_Rug,cat__Product3_Set,cat__Product3_Shoes,cat__Product3_Smartphone,cat__Product3_Smartwatch,cat__Product3_Speaker,cat__Product3_Sweater,cat__Product3_T-Shirt,cat__Product3_Textbook,cat__Product3_Vase
0,0.251910,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.342071,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.494014,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.533562,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.283311,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,0.093074,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
850,0.102799,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
857,0.003881,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
876,0.119653,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [114]:
similarity_matrix = cosine_similarity(df_encoded)
similarity_df = pd.DataFrame(similarity_matrix, index=new['CustomerID'], columns=new['CustomerID'])
similarity_df

CustomerID,C0199,C0146,C0127,C0087,C0070,C0188,C0195,C0008,C0157,C0130,...,C0031,C0198,C0030,C0196,C0115,C0058,C0095,C0151,C0078,C0110
CustomerID,,,,,,,,,,,,,,,,,,,,,
C0199,1.000000,0.404964,0.504705,0.308096,0.503519,0.309001,0.306064,0.210614,0.602466,0.400352,...,0.105298,0.600593,0.205618,0.208191,0.305124,0.201618,0.401106,0.398834,0.102615,0.300293
C0146,0.404964,1.000000,0.507742,0.508065,0.207643,0.314029,0.209632,0.117592,0.404831,0.499297,...,0.305414,0.202612,0.009525,0.309955,0.306942,0.102541,0.400964,0.298392,0.302113,0.399361
C0127,0.504705,0.507742,1.000000,0.317954,0.308996,0.225191,0.410005,0.319834,0.307497,0.398434,...,0.109975,0.203075,0.112084,0.314369,0.211056,0.004541,0.301263,0.296594,0.104567,0.200064
C0087,0.308096,0.508065,0.317954,1.000000,0.309485,0.613233,0.312589,0.127670,0.307847,0.496494,...,0.307341,0.203121,0.211173,0.413118,0.309963,0.202022,0.301068,0.098810,0.301898,0.199867
C0070,0.503519,0.207643,0.308996,0.309485,1.000000,0.115865,0.207941,0.310151,0.404131,0.400231,...,0.105944,0.401380,0.206322,0.209338,0.206614,0.201741,0.201996,0.199312,0.401493,0.101026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C0058,0.201618,0.102541,0.004541,0.202022,0.201741,0.201683,0.201950,0.201906,0.301249,0.100566,...,0.301226,0.300740,0.301297,0.301112,0.201779,1.000000,0.300668,0.099993,0.100998,0.100418
C0095,0.401106,0.400964,0.301263,0.301068,0.201996,0.300095,0.202275,0.202455,0.500750,0.300471,...,0.401097,0.300823,0.102432,0.103392,0.500597,0.300668,1.000000,0.399828,0.101104,0.200392
C0151,0.398834,0.298392,0.296594,0.098810,0.199312,0.098103,0.397621,0.295053,0.299189,0.299957,...,0.199540,0.199913,0.298930,0.297474,0.099683,0.099993,0.399828,1.000000,0.199903,0.199994


In [115]:
similarity_df.loc['C0115']

CustomerID
C0199    0.305124
C0146    0.306942
C0127    0.211056
C0087    0.309963
C0070    0.206614
           ...   
C0058    0.201779
C0095    0.500597
C0151    0.099683
C0078    0.301999
C0110    0.200606
Name: C0115, Length: 199, dtype: float64

In [116]:
results = {}

for customer in new['CustomerID']:
    similar_customers = similarity_df[customer].sort_values(ascending=False).iloc[1:4]  # Exclude self
    results[customer] = list(zip(similar_customers.index, similar_customers.values))

results

{'C0199': [('C0114', 0.6033536112922431),
  ('C0069', 0.6032445863457008),
  ('C0157', 0.6024659824069594)],
 'C0146': [('C0123', 0.6006251531778366),
  ('C0054', 0.5086003495701384),
  ('C0093', 0.508538832691681)],
 'C0127': [('C0051', 0.5134715869947222),
  ('C0146', 0.5077424999099479),
  ('C0026', 0.5070495308454868)],
 'C0087': [('C0188', 0.6132326158216076),
  ('C0155', 0.6126720842354043),
  ('C0102', 0.5156215062172552)],
 'C0070': [('C0118', 0.5057653511626984),
  ('C0154', 0.5056956864385067),
  ('C0041', 0.5056410502111304)],
 'C0188': [('C0028', 0.8074258166475489),
  ('C0087', 0.6132326158216076),
  ('C0190', 0.6029037432545584)],
 'C0195': [('C0175', 0.5092807364526771),
  ('C0184', 0.5090667590806406),
  ('C0116', 0.5077842987750045)],
 'C0008': [('C0098', 0.6125899627963757),
  ('C0079', 0.6069805620608096),
  ('C0024', 0.5153665732226171)],
 'C0157': [('C0069', 0.6031297868667298),
  ('C0199', 0.6024659824069594),
  ('C0039', 0.5038597707197661)],
 'C0130': [('C0094',

In [117]:
lookalikes = pd.DataFrame.from_dict(results, orient='index', columns=['Lookalike_1', 'Lookalike_2', 'Lookalike_3'])
lookalikes20=lookalikes.iloc[:20,:]
lookalikes20

,Lookalike_1,Lookalike_2,Lookalike_3
C0199,"(C0114, 0.6033536112922431)","(C0069, 0.6032445863457008)","(C0157, 0.6024659824069594)"
C0146,"(C0123, 0.6006251531778366)","(C0054, 0.5086003495701384)","(C0093, 0.508538832691681)"
C0127,"(C0051, 0.5134715869947222)","(C0146, 0.5077424999099479)","(C0026, 0.5070495308454868)"
C0087,"(C0188, 0.6132326158216076)","(C0155, 0.6126720842354043)","(C0102, 0.5156215062172552)"
C0070,"(C0118, 0.5057653511626984)","(C0154, 0.5056956864385067)","(C0041, 0.5056410502111304)"
C0188,"(C0028, 0.8074258166475489)","(C0087, 0.6132326158216076)","(C0190, 0.6029037432545584)"
C0195,"(C0175, 0.5092807364526771)","(C0184, 0.5090667590806406)","(C0116, 0.5077842987750045)"
C0008,"(C0098, 0.6125899627963757)","(C0079, 0.6069805620608096)","(C0024, 0.5153665732226171)"
C0157,"(C0069, 0.6031297868667298)","(C0199, 0.6024659824069594)","(C0039, 0.5038597707197661)"
C0130,"(C0094, 0.5003385852673299)","(C0146, 0.4992974449212894)","(C0087, 0.4964943661003415)"


In [118]:
lookalikes20.to_csv("Arshad_Jamal_Lookalike.csv")